In [36]:
import pandas as pd
import numpy as np
import json
import requests
import re
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
%matplotlib inline

#Need to use a delay between page scrapes in order to limit getting blocked by Yelp
from time import sleep

In [37]:
#ENTER SEARCH TERMS BELOW:
cuisine_type = ""
location = "New+York,+NY,+United+States"
#"San+Francisco"

#Generate URL based on search terms
base_url = "https://www.yelp.com"
search_url = f"{base_url}/search?find_desc={cuisine_type}&find_loc={location}&attrs=RestaurantsTakeOut%2CRestaurantsDelivery"

#Or manually set search_url by copying directly from Yelp Page if desired
#search_url = "https://www.yelp.com/search?find_desc=burger&find_loc
search_url

'https://www.yelp.com/search?find_desc=&find_loc=New+York,+NY,+United+States&attrs=RestaurantsTakeOut%2CRestaurantsDelivery'

In [38]:
star_container_class = "i-stars__09f24__M1AR7 i-stars--regular-4-half__09f24__wuUkO border-color--default__09f24__NPAKY overflow--hidden__09f24___ayzG"
price_range_class = "lemon--span__373c0__3997G text__373c0__2pB8f priceRange__373c0__2DY87 text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_ text-bullet--after__373c0__1ZHaA"
review_count_class = "reviewCount__09f24__tnBk4 css-chan6m"
next_page_class = "pagination__09f24__VRjN4 border--top__09f24__exYYb border--bottom__09f24___mg5X border-color--default__09f24__NPAKY"
search_result_class = "container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY"


In [39]:
next_page_url = search_url
page_counter = 0
business_list = []

while next_page_url:
    
    request = requests.get(next_page_url)
    soup = BeautifulSoup(request.content,'html.parser')
    
   
    search_results = soup.findAll("div", class_=search_result_class)
    print(f"Page {page_counter}, {len(search_results)-1} results {next_page_url}")
    result_counter = 1

    #
    for search_result in search_results:
        business_info = {}
        try:
            business_name_url = search_result.findAll('a', href=True)[1]
            business_info['url'] = f"https://www.yelp.com{business_name_url['href']}"
            business_info['name'] = business_name_url['name']
            business_info['biz_id'] = business_name_url['href'].split('/biz/')[1].split('?')[0]
        except:
            continue


        try:
            business_info['num_reviews'] = int(re.findall(r"[-+]?\d*\.\d+|\d+",
                                                      search_result.find(
                                                          class_=review_count_class).text )[0] )
        except:
            pass
        try:
            business_info['image_shown'] = search_result.find('img')['src']
        except:
            pass
        
        #Append business information for each search result to a list containing all businesses.
        if business_info:
            business_list.append(business_info)
            
        result_counter+=1
    
    #Set url for next page. If not found, break out of loop.
    page_counter+=10
    if page_counter >190:
        break
    next_page_url = search_url + "&start="+str(page_counter)
    

    
    #Random delay between 2 and 20 seconds to prevent getting blocked
    sleep(np.random.randint(2,20))

print(len(business_list), "businesses scraped")
#business_list

Page 0, 9 results https://www.yelp.com/search?find_desc=&find_loc=New+York,+NY,+United+States&attrs=RestaurantsTakeOut%2CRestaurantsDelivery
Page 10, 9 results https://www.yelp.com/search?find_desc=&find_loc=New+York,+NY,+United+States&attrs=RestaurantsTakeOut%2CRestaurantsDelivery&start=10
Page 20, 9 results https://www.yelp.com/search?find_desc=&find_loc=New+York,+NY,+United+States&attrs=RestaurantsTakeOut%2CRestaurantsDelivery&start=20
Page 30, 9 results https://www.yelp.com/search?find_desc=&find_loc=New+York,+NY,+United+States&attrs=RestaurantsTakeOut%2CRestaurantsDelivery&start=30
Page 40, 9 results https://www.yelp.com/search?find_desc=&find_loc=New+York,+NY,+United+States&attrs=RestaurantsTakeOut%2CRestaurantsDelivery&start=40
Page 50, 9 results https://www.yelp.com/search?find_desc=&find_loc=New+York,+NY,+United+States&attrs=RestaurantsTakeOut%2CRestaurantsDelivery&start=50
Page 60, 9 results https://www.yelp.com/search?find_desc=&find_loc=New+York,+NY,+United+States&attrs=Res

In [40]:
business_info_df = pd.DataFrame(business_list)
#Drop businesses with no reviews
business_info_df.dropna(subset=['num_reviews'], inplace=True)
#Drop duplicates
business_info_df.drop(business_info_df[business_info_df.biz_id.duplicated(keep='first')].index, inplace=True)
print(len(business_info_df))
business_info_df.tail()

200


,url,name,biz_id,num_reviews,image_shown
195,https://www.yelp.com/biz/maison-pickle-new-york,Maison Pickle,maison-pickle-new-york,1353,https://s3-media0.fl.yelpcdn.com/bphoto/c14z3u...
196,https://www.yelp.com/biz/pokeworks-new-york-12,Pokeworks,pokeworks-new-york-12,1761,https://s3-media0.fl.yelpcdn.com/bphoto/vel7my...
197,https://www.yelp.com/biz/cuba-restaurant-and-r...,Cuba Restaurant and Rum Bar,cuba-restaurant-and-rum-bar-new-york,1617,https://s3-media0.fl.yelpcdn.com/bphoto/muPAyv...
198,https://www.yelp.com/biz/freemans-new-york-3,Freemans,freemans-new-york-3,2131,https://s3-media0.fl.yelpcdn.com/bphoto/u8-TgA...
199,https://www.yelp.com/biz/moms-kitchen-and-bar-...,Mom’s Kitchen & Bar,moms-kitchen-and-bar-new-york,1162,https://s3-media0.fl.yelpcdn.com/bphoto/2em4dL...


In [41]:
# #CLEAN UP BUSINESS NAMES
# business_names = []
# for name in business_info_df.name:
#     name = name.replace('â\x80\x99',"\'")
#     business_names.append(name)
# business_info_df.name = business_names

In [42]:
def get_reviews(business_name, business_index, yelp_business_url, verbose=False):

    next_page_url = yelp_business_url+"?start=0"

    reviews_list = []
    page_counter=0

   
    while next_page_url:
        # if verbose:
        #    print(f"Page {page_counter}, {next_page_url}")

        
        request = requests.get(next_page_url)
        soup = BeautifulSoup(request.content,'html.parser')
        

        reviews_list.extend(get_reviews_page_v1(soup,business_name,business_index,verbose))
        
        
        if page_counter >80:
            break

        page_counter+=10
        next_page_url = yelp_business_url+"?start=" +str(page_counter)
        
    
        # if soup.find("link", attrs={'rel':'next'}):
        #     next_page_url = soup.find("link", attrs={'rel':'next'})['href']
        #     page_counter+=1
        # else:
        #     break
        
        #Random delay between 1 and 4 seconds to prevent getting blocked
        sleep(np.random.randint(1,3))
    
    return reviews_list

In [43]:
def get_reviews_page_v1(soup, business_name, business_index, verbose=False):
    
    search_result_class = "review__09f24__oHr9V border-color--default__09f24__NPAKY"
    
    #Get each review block
    reviews = soup.findAll(class_=search_result_class)
    reviews_list=[]
    skipped_review_counter=0

    for review in reviews:
        try:
            review_info = {}
            review_info["business_name"] = business_name
            review_info["business_index"] = business_index
            review_info["review"] = review.find("span", attrs={"class": "raw__09f24__T4Ezm", "lang": "en"}).text
           
            try:
                review_info["user_id"] = review.find('a')['href'].split('userid=')[1]
            except:
                None

            reviews_list.append(review_info)
        except:
            skipped_review_counter+=1
            
    if verbose:
        if skipped_review_counter!=0:
            print(f"Skipped {skipped_review_counter} reviews")

    return(reviews_list)

In [44]:
#TESTING SCRAPER FOR A SINGLE BUSINESS
index_num = 4
business_url = business_info_df.url[index_num]
business_name = business_info_df.name[index_num]
business_index = business_info_df.biz_id[index_num]

reviews_df = pd.DataFrame(get_reviews(business_name,business_index, business_url,verbose=True))

In [45]:
business_info_df.shape

(200, 5)

In [46]:
# reviews_df = pd.DataFrame()
# for index_num in range(10):
#     business_url = business_info_df.url[index_num]
#     business_name = business_info_df.name[index_num]
#     business_index = business_info_df.biz_id[index_num]

#     df = pd.DataFrame(get_reviews(business_name,business_index, business_url,verbose=True))
#     reviews_df = reviews_df.append(df)
# reviews_df

In [47]:
last_index_completed = 0
all_reviews = []
while last_index_completed < len(business_info_df):
    biz = business_info_df.iloc[last_index_completed]
    
    #Random delay between 1 and 5 seconds to try to prevent getting blocked
    sleep(np.random.randint(2,5))
    print(last_index_completed, int(biz['num_reviews']),"reviews\t", biz['name'], biz['url'], end='')
    
    #Get all reviews for the business and add to list.
    all_reviews.extend(get_reviews(biz['name'],biz['biz_id'],biz['url'])) 

    print(" completed")
    last_index_completed+=1

0 8767 reviews	 Levain Bakery https://www.yelp.com/biz/levain-bakery-new-york completed
1 13407 reviews	 Katz’s Delicatessen https://www.yelp.com/biz/katzs-delicatessen-new-york completed
2 5184 reviews	 Bibble & Sip https://www.yelp.com/biz/bibble-and-sip-new-york-2 completed
3 5843 reviews	 LoveMama https://www.yelp.com/biz/lovemama-new-york completed
4 3833 reviews	 Best Bagel & Coffee https://www.yelp.com/biz/best-bagel-and-coffee-new-york completed
5 4034 reviews	 Thai Villa https://www.yelp.com/biz/thai-villa-new-york-2 completed
6 4249 reviews	 Prince Street Pizza https://www.yelp.com/biz/prince-street-pizza-new-york-2 completed
7 5581 reviews	 Burger & Lobster - Flatiron NYC https://www.yelp.com/biz/burger-and-lobster-flatiron-nyc-new-york completed
8 3063 reviews	 Gramercy Tavern https://www.yelp.com/biz/gramercy-tavern-new-york completed
9 6209 reviews	 Peter Luger https://www.yelp.com/biz/peter-luger-brooklyn-2 completed
10 3309 reviews	 Doughnut Plant https://www.yelp.com/b

In [48]:
# for index_num in range(20,30):
#     business_url = business_info_df.url[index_num]
#     business_name = business_info_df.name[index_num]
#     business_index = business_info_df.biz_id[index_num]

#     df = pd.DataFrame(get_reviews(business_name,business_index, business_url,verbose=True))
#     reviews_df = reviews_df.append(df)


In [49]:
all_reviews_df = pd.DataFrame(all_reviews)
all_reviews_df

,business_name,business_index,review,user_id
0,Levain Bakery,levain-bakery-new-york,"Putting in my $0.02 on Levain among the 5,000+...",YX-uSqwvb1uqixWbXPgvLw
1,Levain Bakery,levain-bakery-new-york,"Of course they are good. Get them, try them, s...",dRq6UysiItuIoZEhw8UHrg
2,Levain Bakery,levain-bakery-new-york,This is the second time I've tried these cooki...,V1swFWt_8ofnG_aj0Sf3tw
3,Levain Bakery,levain-bakery-new-york,"What can I say? It's Levain's cookies, everyon...",gDeacL_V-4Pjb0jVHOpT7w
4,Levain Bakery,levain-bakery-new-york,We had these cookies mailed to us... from my h...,j9u5AkkYhcxcsvwUxS7G0g
...,...,...,...,...
9875,Mom’s Kitchen & Bar,moms-kitchen-and-bar-new-york,This place is SO fun! The food isn't like anyt...,RImnKNmgSmA8agGLkKh5Kw
9876,Mom’s Kitchen & Bar,moms-kitchen-and-bar-new-york,"The wait was 45 mins long, but understandable ...",xwP3PWNNwXPBU6i7UOU34Q
9877,Mom’s Kitchen & Bar,moms-kitchen-and-bar-new-york,Came here for Sunday brunch at 1:00 with a gir...,RVCrt1QmwsHOFJi6IOWMtg
9878,Mom’s Kitchen & Bar,moms-kitchen-and-bar-new-york,"Delicious food, friendly staff, super cool env...",hb94wEmpKNU7XqDT-qgCmA


In [50]:
#business_info_df.to_csv('csv_files/business_info_all.csv',index=False)
#all_reviews_df.to_csv('reviews_SanFrancisco.csv',index=False)